In [15]:
import pony.orm as orm
import os
import json

In [16]:
database="media.sqlite"

In [17]:
if os.path.exists(database):
    os.unlink(database)

In [18]:
def get_xxhash(fname):
    hash64 = xxhash.xxh64()
    with open(fname, "rb") as f:
        for chunk in iter(lambda: f.read(4096), b""):
            hash64.update(chunk)
    return hash64.hexdigest()

In [19]:
# Database of receipts.
filename = os.path.abspath(database)
db = orm.Database(
    provider='sqlite',
    filename=filename,
    create_db=True,
)
class Media(db.Entity):
    xxhash = orm.Required(str)
    size   = orm.Required(int)
    exif = orm.Required(str)

class MediaPath(db.Ent):
    
    
db.generate_mapping(create_tables=True)

In [20]:
import os 
import exiftool
import get_files
import xxhash
import py.path
import_dir = "/pictures/2018-03-02T16:00:01-05:00/DCIM"

In [21]:
IMAGE_EXT = [".cr2", ".dng", ".jpg"]
VIDEO_EXT = [".mp4"]

In [22]:
et = exiftool.ExifTool()
et.start()

In [23]:
media_files = get_files.get_files(import_dir, decorate=py.path.local)
for media_file in media_files:
    if media_file.ext.lower() in IMAGE_EXT:
        EXIF = et.get_metadata(str(media_file))
        Media(
            path = str(media_file),
            exif = json.dumps(EXIF),
            xxhash = str(media_file),
            size   = media_file.size(),
        )
        db.commit()